# Task 4: Demonstrating Quantum Advantage

In [ ]:
"""
This task is intentionally open-ended, so no boilerplate code is provided.

You may use this notebook to develop your solution, or create a separate file if you prefer.
We recommend starting by copying over your previous implementations of the QRNG, TRNG, and PRNG.
Then, explore ways to modularize and combine these components to design a use case that
demonstrates the unique advantages of quantum randomness.

Your write-up can be included directly in this notebook or submitted separately.
You're welcome to prepare it as a Google Doc or LaTeX document and upload a PDF to the GitHub repository—
just be sure to clearly indicate where it can be found if it's not included here.
"""

In [6]:
%pip install requests
%pip install numpy
%pip install matplotlib
%pip install pyaudio
%pip install psutil
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      /private/var/folders/f5/299vphyd6fz0xl9_z3gz0w7h0000gn/T/pip-build-env-vd_vx0di/overlay/lib/python3.12/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI

In [16]:
# Package Imports - feel free to add what you think might be useful! 
import requests
import time
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import hashlib
import struct
import psutil
import platform
import math
import pandas as pd

In [17]:
def get_qrng_floats(n, min_val=-32768, max_val=32767):
    if n > 128:
        n = 128
    url = "https://qrng.idqloud.com/api/1.0/short"
    headers = {"X-API-KEY": "aTo4BKRvnc49uRWDk034zaua87vGRXKk9TMLdfkI"}
    params = {"min": min_val, "max": max_val, "quantity": n}
    response = requests.get(url, headers=headers, params=params)
    raw = np.array(response.json()['data'], dtype=np.float64)
    scaled = raw - min_val
    scaled = scaled / (abs(max_val) + abs(min_val)) 

    #Random numbers scale to [0,1)
    return scaled

In [18]:
def get_laplace_transform(P, epsilon):
    # P is the scaled random numbers from get_qrng_floats()
    # Epsilon is the threshold for privacy (smaller epsilon = more privacy = less utility)
    b = 1 / epsilon
    laplace = []
    for p in P:
        if p == 0:
            laplace.append(float('-inf'))
        elif p < 0.5:
            laplace.append(b * math.log(2 * p))
        else:
            laplace.append(-b * math.log(2 * (1 - p)))
    return laplace

In [ ]:
random_numbers = get_qrng_floats(40)
print(random_numbers)


[0.90946822 0.22011139 0.95022507 0.51241321 0.5309987  0.5803006
 0.27533379 0.50621805 0.18062104 0.32956435 0.69973297 0.69056229
 0.69633021 0.52645151 0.96362249 0.86045624 0.32326238 0.98597696
 0.20048829 0.13127337 0.11215381 0.23535515 0.6030518  0.69616236
 0.45914397 0.26675822 0.5918822  0.18544289 0.23683528 0.60091554
 0.89181353 0.57206073 0.89750515 0.76948196 0.35817502 0.13661402
 0.59253834 0.93873503 0.78428321 0.72646677]


In [47]:
epsilon = 0.25
laplace_transform = get_laplace_transform(random_numbers, epsilon)

In [32]:
size = 20

changes = -np.random.poisson(2, size)
positive = 324 + np.maximum.reduce([changes, np.random.normal(0, 4, size).astype(int)]).cumsum()
total = 100000 + changes.cumsum()

In [33]:
hiv_df = pd.DataFrame(np.array([positive, total]).T, columns=["Positive", "Total"])
hiv_df.index = range(2000, 2020)

In [55]:
display(hiv_df)

,Positive,Total
2000,325,99999
2001,324,99996
2002,329,99995
2003,330,99995
2004,336,99993
2005,337,99991
2006,337,99987
2007,334,99984
2008,331,99980
2009,332,99977


In [56]:
hiv_df_differential_privacy = hiv_df.copy(deep=True)
hiv_df_differential_privacy["Positive"] = hiv_df["Positive"] + np.array(laplace_transform[:20]).astype(int)
hiv_df_differential_privacy["Total"] = hiv_df["Total"] + np.array(laplace_transform[20:]).astype(int)

display(hiv_df_differential_privacy)

,Positive,Total
2000,331,99994
2001,321,99993
2002,338,99995
2003,330,99996
2004,336,99993
2005,337,99989
2006,335,99987
2007,334,99981
2008,327,99978
2009,331,99977
